In [1]:
#!pip install -u geopandas
#!pip install -u regionmask

In [2]:
# Loading libraries
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import xarray as xr
import xesmf as xe
import dask
import season_util as su
import jetlatcalcs as jlat
import mask

you have successfully imported the jet latitude calculation subroutines


In [3]:
import warnings
warnings.filterwarnings("ignore")

#### Find/read ts data:

In [4]:
#historical simulations
import intake
var1 = ["ts"]
model=['AWI-CM-1-MR','BCC-CSM2-MR','CAMS-CSM1-0', 'CanESM5','MIROC-ES2L','MIROC6','UKESM1-0-LL','MRI-ESM2-0']
col = intake.open_esm_datastore("/glade/collections/cmip/catalog/intake-esm-datastore/catalogs/glade-cmip6.json")
cat = col.search(activity_id="CMIP",experiment_id="historical", variable_id=var1,
table_id="Amon", grid_label="gn", source_id=model )
dset_dict_hist_ts = cat.to_dataset_dict(cdf_kwargs={'chunks': {"time": 36}})

--> The keys in the returned dictionary of datasets are constructed as follows:
	'activity_id.institution_id.source_id.experiment_id.table_id.grid_label'

--> There will be 7 group(s)


In [5]:
#ssp simulations
col = intake.open_esm_datastore("/glade/collections/cmip/catalog/intake-esm-datastore/catalogs/glade-cmip6.json")
cat = col.search(activity_id="ScenarioMIP",experiment_id="ssp370", variable_id=var1,
table_id="Amon", grid_label="gn", source_id=model )
dset_dict_ssp370_ts = cat.to_dataset_dict(cdf_kwargs={'chunks': {"time": 36}})

--> The keys in the returned dictionary of datasets are constructed as follows:
	'activity_id.institution_id.source_id.experiment_id.table_id.grid_label'

--> There will be 5 group(s)


#### Calculate Nino3.4 time series:

In [6]:
#subset for time period (1948 through 2014)
hist_ts = {}
for key, ds in dset_dict_hist_ts.items():
    hist_ts[key] = ds.sel(time = slice('1948-01-01','2015-01-01'))

In [7]:
from nino34 import nino34

In [8]:
# usage: nino = nino34(da,'ts','DJF')
hist_nino = {}
for key in hist_ts:
    print(key)
    hist_nino[key] = nino34(hist_ts[key], 'ts', 'DJF')
    ## add days per month weighting in DJF calc

CMIP.BCC.BCC-CSM2-MR.historical.Amon.gn
CMIP.CAMS.CAMS-CSM1-0.historical.Amon.gn
CMIP.CCCma.CanESM5.historical.Amon.gn
CMIP.MIROC.MIROC-ES2L.historical.Amon.gn
CMIP.MIROC.MIROC6.historical.Amon.gn
CMIP.MOHC.UKESM1-0-LL.historical.Amon.gn
CMIP.MRI.MRI-ESM2-0.historical.Amon.gn


In [19]:
#hist_nino

{'CMIP.BCC.BCC-CSM2-MR.historical.Amon.gn': <xarray.DataArray (member_id: 3, year: 67)>
 dask.array<truediv, shape=(3, 67), dtype=float64, chunksize=(1, 1), chunktype=numpy.ndarray>
 Coordinates:
   * member_id  (member_id) <U8 'r2i1p1f1' 'r1i1p1f1' 'r3i1p1f1'
   * year       (year) int64 1948 1949 1950 1951 1952 ... 2011 2012 2013 2014,
 'CMIP.CAMS.CAMS-CSM1-0.historical.Amon.gn': <xarray.DataArray (member_id: 2, year: 67)>
 dask.array<truediv, shape=(2, 67), dtype=float64, chunksize=(1, 1), chunktype=numpy.ndarray>
 Coordinates:
   * member_id  (member_id) <U8 'r2i1p1f1' 'r1i1p1f1'
   * year       (year) int64 1948 1949 1950 1951 1952 ... 2011 2012 2013 2014,
 'CMIP.CCCma.CanESM5.historical.Amon.gn': <xarray.DataArray (member_id: 44, year: 67)>
 dask.array<truediv, shape=(44, 67), dtype=float64, chunksize=(1, 1), chunktype=numpy.ndarray>
 Coordinates:
   * member_id  (member_id) <U9 'r2i1p1f1' 'r5i1p1f1' ... 'r18i1p1f1' 'r6i1p1f1'
   * year       (year) int64 1948 1949 1950 1951 1952

#### Find/read pr data:

In [10]:
#historical simulations
import intake
var2 = ["pr"]
model=['AWI-CM-1-MR','BCC-CSM2-MR','CAMS-CSM1-0', 'CanESM5','MIROC-ES2L','MIROC6','UKESM1-0-LL','MRI-ESM2-0']
col = intake.open_esm_datastore("/glade/collections/cmip/catalog/intake-esm-datastore/catalogs/glade-cmip6.json")
cat = col.search(activity_id="CMIP",experiment_id="historical", variable_id=var2,
table_id="Amon", grid_label="gn", source_id=model )
dset_dict_hist = cat.to_dataset_dict(cdf_kwargs={'chunks': {"time": 36}})

--> The keys in the returned dictionary of datasets are constructed as follows:
	'activity_id.institution_id.source_id.experiment_id.table_id.grid_label'

--> There will be 7 group(s)


In [11]:
#ssp simulations
col = intake.open_esm_datastore("/glade/collections/cmip/catalog/intake-esm-datastore/catalogs/glade-cmip6.json")
cat = col.search(activity_id="ScenarioMIP",experiment_id="ssp370", variable_id=var2,
table_id="Amon", grid_label="gn", source_id=model )
dset_dict_ssp370 = cat.to_dataset_dict(cdf_kwargs={'chunks': {"time": 36}})

--> The keys in the returned dictionary of datasets are constructed as follows:
	'activity_id.institution_id.source_id.experiment_id.table_id.grid_label'

--> There will be 7 group(s)


In [12]:
#subset for time period (1948 through 2014)
hist = {}
for key, ds in dset_dict_hist.items():
    hist[key] = ds.sel(time = slice('1948-01-01','2015-01-01'))

In [13]:
## check attributes of pr variable. units are kg m-2 s-1
#dset_dict_hist['CMIP.BCC.BCC-CSM2-MR.historical.Amon.gn'].pr

In [14]:
## testing mask
#ds = hist['CMIP.BCC.BCC-CSM2-MR.historical.Amon.gn']
#ca_mask = mask.state_mask(mask.read_state("../data/states.shp", 'California'), ds, 'California', 'CA')
#ds.where(ca_mask == 1)

#### Calculate DJF precip time series for California:

In [15]:
hist_ca_pr = {}
for key in hist:
    print(key)
    ca_mask = mask.state_mask(mask.read_state("../data/states.shp", 'California'), hist[key], 'California', 'CA')
    hist_ca_pr[key] = su.season_ts((hist[key].where(ca_mask == 1)), "pr", "DJF").mean(dim=('lat', 'lon'))
## add days per month weighting and areacella weighting

CMIP.BCC.BCC-CSM2-MR.historical.Amon.gn
CMIP.CAMS.CAMS-CSM1-0.historical.Amon.gn
CMIP.CCCma.CanESM5.historical.Amon.gn
CMIP.MIROC.MIROC-ES2L.historical.Amon.gn
CMIP.MIROC.MIROC6.historical.Amon.gn
CMIP.MOHC.UKESM1-0-LL.historical.Amon.gn
CMIP.MRI.MRI-ESM2-0.historical.Amon.gn


In [21]:
# look at ca precip time series
#hist_ca_pr

#### Detrend time series

#### Calculate interannual correlation between Nino3.4 and CA DJF precip: 

#### Calculate mean correlation across ensembles members within each model:

In [17]:
#hist_emean = {}
#for key, ds in dset_dict_hist.items():
#    hist_mean[key] = ds.sel(time = slice('1979-01-01','2006-01-01')).mean(dim=["member_id"])
    
#ssp370_emean = {}
#for key, ds in dset_dict_ssp370.items():
#ssp370_mean[key] = ds.sel(time = slice('2070-01-01','2099-01-01')).mean(dim=["member_id"])

#### Calculate mean DJF precip in historical and future periods: 

In [18]:
### ADD TIME SUBSETTING!!!!
hist_ca_mean = {}
for key in hist{}[0]:
    ca_mask = mask.state_mask(mask.read_state("../data/states.shp", 'California'), hist_emean[0][key], 'California', 'CA')
    hist_ca_mean[key] = su.season_mean((hist_emean[0][key]*ca_mask), var="pr", season="DJF").mean(dim=('lat', 'lon'))

ssp370_ca_mean = {}
for key in ssp370_emean[0]:
    ca_mask = mask.state_mask(mask.read_state("../data/states.shp", 'California'), ssp370_emean[0][key], 'California', 'CA')
    ssp370_ca_mean[key] = su.season_mean((ssp370_emean[0][key]*ca_mask), var="pr", season="DJF").mean(dim=('lat', 'lon'))

NameError: name 'hist_emean' is not defined

### Plot something: